In [ ]:
!pip install numpy
!pip install scikit-learn
!pip install pandas
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Load Dataset

In [ ]:
!wget http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
!tar xvzf review_polarity.tar.gz

from sklearn.datasets import load_files  
movies = load_files(container_path="txt_sentoken", encoding="utf-8")  
# print(movies['DESCR'])
x, y = movies.data, movies.target

--2022-11-13 19:32:19--  http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/x-gzip]
Saving to: ‘review_polarity.tar.gz’

review_polarity.tar 100%[===================>]   2.98M  --.-KB/s    in 0.1s    

2022-11-13 19:32:20 (22.1 MB/s) - ‘review_polarity.tar.gz’ saved [3127238/3127238]

txt_sentoken/neg/cv000_29416.txt
txt_sentoken/neg/cv001_19502.txt
txt_sentoken/neg/cv002_17424.txt
txt_sentoken/neg/cv003_12683.txt
txt_sentoken/neg/cv004_12641.txt
txt_sentoken/neg/cv005_29357.txt
txt_sentoken/neg/cv006_17022.txt
txt_sentoken/neg/cv007_4992.txt
txt_sentoken/neg/cv008_29326.txt
txt_sentoken/neg/cv009_29417.txt
txt_sentoken/neg/cv010_29063.txt
txt_sentoken/neg/cv011_13044.txt
txt_sentoken/neg/cv012_29411.txt
txt_sentoken/neg/cv0

In [ ]:
print(type(x[0]))
print(type(y))
print(y.shape)

<class 'str'>
<class 'numpy.ndarray'>
(2000,)


# Preprocess Dataset

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')


from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
import re

stemmer = WordNetLemmatizer()
docs = []

for doc in x: 
    
    # Remove all stopwords
    stop_words = stopwords.words('english')
    stopwords_dict = Counter(stop_words) # this is just for optimazation purposes
    document = ' '.join([word for word in doc.split() if word not in stopwords_dict])

    
    # Split the document based on whitespaces (--> List of words)
    document = document.split()

    # Lemmatization
    document = [stemmer.lemmatize(word) for word in document]

    # Reconstruct the document by joining the words on each whitespace
    document = ' '.join(document)


    # Convert to Lowercase
    document = document.lower()

    # Remove stopwords that occurered after lemmatization
    document = ' '.join([word for word in document.split() if word not in stopwords_dict])

   
    # Remove non-word (special) characters such as punctuation
    document = re.sub(r'\W', ' ', document)

    # Remove all single characters
    document = re.sub(r'\s+[a-z]\s+', ' ', document)

    # Remove once again stopwords that occured
    document = ' '.join([word for word in document.split() if word not in stopwords_dict])

      
    # Remove numbers
    document = re.sub(r'\s+[0-9]+', ' ', document)


    # Remove honorifics
    document = re.sub(r'\s+mr\s+|\s+dr\s+|\s+jr\s+|\s+ms\s+|\s+miss\s+', ' ', document)

    # Substitute multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Append all documents into a list 'docs'
    docs.append(document)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(docs[0])

arnold schwarzenegger icon action enthusiast since late lately film sloppy one liner getting worse hard seeing arnold freeze batman robin especially say ton ice joke hey got million matter arnold signed another expensive blockbuster compare like terminator series true lie even eraser called dark thriller devil gabriel byrne come upon earth impregnate woman robin tunney happens every year basically destroy world apparently god chosen one man one man jericho cane arnold help trusty sidekick kevin pollack stop nothing let devil take world part actually absurd would fit right dogma yes film weak better blockbuster right sleepy hollow make world enough look like star film anyway definitely seem like arnold movie type film see sure gave chuckle well known one liner seemed confused character film going understandable especially ending changed according source aside form still walked much like past film sorry say arnold maybe end action day speaking action film hardly explosion fight devil mad

# Split Dataset into train(70%), development(20%) and test(10%)

In [ ]:
from sklearn.model_selection import train_test_split  

x_train, x_temp, y_train, y_temp = train_test_split(docs, y, test_size=0.3, random_state=17)
x_dev, x_test, y_dev, y_test = train_test_split(x_temp, y_temp, test_size=0.333, random_state=25)

print(len(y_train), len(y_dev), len(y_test))
print(x_train[0])

1400 400 200
many people know james whale safe bet good chunk seen movie believe semi biographical god monster whale would wanted way insightful haunting exploration last day frankenstein bride frankenstein director notable introducing one first complicated gay character hollywood movie god monster interest biopic whale life track final day life probably better movie focus whale ian mckellen untraditional deceptive lust heterosexual gardener clayton boone brendan fraser begin whale fairly talented artist aside director asking boone sit pose portrait first boone realize whale gay grows fascinated old man discover whale sexual orientation dedicated protective maid lynn redgrave frightened refusing sit guy come back storm disgusted whale locker room talk boone return yet another time whale promise tone aforementioned locker room talk find platonic relationship strengthening meanwhile jimmy suffering hallucination mental attack result stroke long ago ailment lead desperation depression bui

# Feature exctraction (Boolean) based on Mutual Information a.k.a Information Gain

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Use unigram & bi-gram tf*idf features
boolean_vectorizer = CountVectorizer(
    ngram_range = (1, 1), 
    max_features = 5000,
    min_df = 10,
    binary = True
)

# Fit Vectorizer on train data
# Transform on all data (train - dev - test)
x_train_boolean = boolean_vectorizer.fit_transform(x_train)
x_dev_boolean = boolean_vectorizer.transform(x_dev)
x_test_boolean = boolean_vectorizer.transform(x_test)

print(boolean_vectorizer.get_feature_names())

['abandon', 'abandoned', 'ability', 'able', 'aboard', 'absence', 'absent', 'absolute', 'absolutely', 'absurd', 'abuse', 'abusive', 'abyss', 'academy', 'accent', 'accept', 'acceptable', 'accepted', 'accepts', 'accident', 'accidentally', 'acclaimed', 'accompanied', 'accompanying', 'accomplish', 'accomplished', 'accomplishment', 'according', 'account', 'accuracy', 'accurate', 'accurately', 'accused', 'ace', 'achieve', 'achieved', 'achievement', 'achieves', 'acid', 'across', 'act', 'acted', 'acting', 'action', 'activity', 'actor', 'actors', 'actress', 'actual', 'actually', 'ad', 'adam', 'adaptation', 'adapted', 'add', 'added', 'addicted', 'addiction', 'adding', 'addition', 'address', 'adept', 'adequate', 'adjective', 'admirable', 'admirably', 'admire', 'admired', 'admission', 'admit', 'admits', 'admittedly', 'adolescent', 'adorable', 'adult', 'advance', 'advanced', 'advantage', 'adventure', 'advice', 'advised', 'advocate', 'affair', 'affect', 'affected', 'affecting', 'affection', 'affleck'

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print('Shape of training data: {}'.format(x_train_boolean.shape))
print('Shape of developemnt data: {}\n'.format(x_dev_boolean.shape))
print('Shape of test data: {}\n'.format(x_test_boolean.shape))
print(x_test_boolean[0, :])  # Sparce matrix --> Saves only the cells which are not zero

Shape of training data: (1400, 5000)
Shape of developemnt data: (400, 5000)

Shape of test data: (200, 5000)

  (0, 43)	1
  (0, 51)	1
  (0, 74)	1
  (0, 138)	1
  (0, 142)	1
  (0, 156)	1
  (0, 182)	1
  (0, 195)	1
  (0, 196)	1
  (0, 231)	1
  (0, 291)	1
  (0, 293)	1
  (0, 324)	1
  (0, 354)	1
  (0, 355)	1
  (0, 395)	1
  (0, 396)	1
  (0, 399)	1
  (0, 449)	1
  (0, 493)	1
  (0, 494)	1
  (0, 524)	1
  (0, 536)	1
  (0, 584)	1
  (0, 635)	1
  :	:
  (0, 4121)	1
  (0, 4159)	1
  (0, 4201)	1
  (0, 4221)	1
  (0, 4238)	1
  (0, 4252)	1
  (0, 4342)	1
  (0, 4414)	1
  (0, 4453)	1
  (0, 4504)	1
  (0, 4518)	1
  (0, 4528)	1
  (0, 4536)	1
  (0, 4579)	1
  (0, 4639)	1
  (0, 4787)	1
  (0, 4792)	1
  (0, 4800)	1
  (0, 4859)	1
  (0, 4862)	1
  (0, 4866)	1
  (0, 4962)	1
  (0, 4965)	1
  (0, 4987)	1
  (0, 4989)	1


In [ ]:
import numpy as np
from sklearn.feature_selection import mutual_info_classif

# Number of features to keep
k = 2500

# Convert sparse matrix to np.array --> Slow
x_train_boolean_array = x_train_boolean.toarray()  

# The function relies on nonparametric methods 
# based on entropy estimation from k-nearest neighbors distances
mutual_information = mutual_info_classif(x_train_boolean_array, y_train, n_neighbors=3, 
                                         discrete_features=True,
                                         random_state=42)

# Indexes of the feature columns
indexes = np.arange(len(mutual_information))

# Sort by mutual information values --> (value, idx)
sorted_indexes = sorted(zip(mutual_information, indexes), reverse=True)
print('Sorted indexes: {}'.format(sorted_indexes[:4]))

# keep the indexes of the best k features
best_idxs = [idx for val, idx in sorted_indexes[:k]]
print('Best indexes: {}\n'.format(best_idxs[:4]))

# Keep only the colums of the best features
x_train_best_boolean = x_train_boolean_array[:, best_idxs]
x_test_best_boolean = x_test_boolean[:, best_idxs]

print('Shape of training data: {} --> {}'.format(x_train_boolean.shape, x_train_best_boolean.shape))
print('Shape of test data: {} --> {}\n'.format(x_test_boolean.shape, x_test_best_boolean.shape))

Sorted indexes: [(0.040909811643391, 328), (0.029615420796993516, 4961), (0.02173632403392732, 477), (0.01940558382314584, 316)]
Best indexes: [328, 4961, 477, 316]

Shape of training data: (1400, 5000) --> (1400, 2500)
Shape of test data: (200, 5000) --> (200, 2500)



# Feature exctraction (Tf-Idf) based on Mutual Information a.k.a Information Gain

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Use unigram & bi-gram tf*idf features
vectorizer = TfidfVectorizer(
    ngram_range = (1, 2), 
    max_features = 5000,
    #Apply sublinear tf scaling, i.e. replace tf with 1 + log(tf).
    sublinear_tf = True, 
)


# Fit Vectorizer on train data
# Transform on all data (train - test)
x_train_tfidf = vectorizer.fit_transform(x_train)
x_dev_tfidf = vectorizer.fit_transform(x_dev)
x_test_tfidf = vectorizer.transform(x_test)

print(boolean_vectorizer.get_feature_names())

['abandon', 'abandoned', 'ability', 'able', 'aboard', 'absence', 'absent', 'absolute', 'absolutely', 'absurd', 'abuse', 'abusive', 'abyss', 'academy', 'accent', 'accept', 'acceptable', 'accepted', 'accepts', 'accident', 'accidentally', 'acclaimed', 'accompanied', 'accompanying', 'accomplish', 'accomplished', 'accomplishment', 'according', 'account', 'accuracy', 'accurate', 'accurately', 'accused', 'ace', 'achieve', 'achieved', 'achievement', 'achieves', 'acid', 'across', 'act', 'acted', 'acting', 'action', 'activity', 'actor', 'actors', 'actress', 'actual', 'actually', 'ad', 'adam', 'adaptation', 'adapted', 'add', 'added', 'addicted', 'addiction', 'adding', 'addition', 'address', 'adept', 'adequate', 'adjective', 'admirable', 'admirably', 'admire', 'admired', 'admission', 'admit', 'admits', 'admittedly', 'adolescent', 'adorable', 'adult', 'advance', 'advanced', 'advantage', 'adventure', 'advice', 'advised', 'advocate', 'affair', 'affect', 'affected', 'affecting', 'affection', 'affleck'

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print('Shape of training data: {}'.format(x_train_tfidf.shape))
print('Shape of dev data: {}'.format(x_dev_tfidf.shape))
print('Shape of test data: {}\n'.format(x_test_tfidf.shape))

print(x_test_tfidf[0, :])  # Sparce matrix --> Saves only the cells which are not zero

Shape of training data: (1400, 5000)
Shape of dev data: (400, 5000)
Shape of test data: (200, 5000)

  (0, 4986)	0.04582097159124879
  (0, 4982)	0.04798798078274946
  (0, 4942)	0.02947077628637877
  (0, 4937)	0.057442992376969024
  (0, 4821)	0.05290796555812057
  (0, 4816)	0.05189430219672476
  (0, 4812)	0.04658151777918544
  (0, 4746)	0.061982039506582695
  (0, 4736)	0.055823756044314464
  (0, 4732)	0.061982039506582695
  (0, 4604)	0.09884893493029144
  (0, 4601)	0.04508886378939776
  (0, 4546)	0.061982039506582695
  (0, 4502)	0.09884893493029144
  (0, 4500)	0.06981585913273293
  (0, 4492)	0.08527600217434278
  (0, 4482)	0.06114865404267766
  (0, 4465)	0.041942445217820146
  (0, 4408)	0.11127906307199317
  (0, 4369)	0.03400278853078032
  (0, 4305)	0.10285004829716128
  (0, 4231)	0.088294519554575
  (0, 4230)	0.05290796555812057
  (0, 4217)	0.0923653263375909
  (0, 4179)	0.04018769552233994
  :	:
  (0, 665)	0.02647539133976096
  (0, 659)	0.06114865404267766
  (0, 625)	0.044732824839540

In [ ]:
# Number of features to keep
k = 2500

# Convert sparse matrix to np.array --> Slow
x_train_tfidf_array = x_train_tfidf.toarray()  

# The function relies on nonparametric methods 
# based on entropy estimation from k-nearest neighbors distances
mutual_information = mutual_info_classif(x_train_tfidf_array, y_train, n_neighbors=3, 
                                         discrete_features=True,
                                         random_state=4321)

# Indexes of the feature columns
indexes = np.arange(len(mutual_information))

# Sort by mutual information values --> (value, idx)
sorted_indexes = sorted(zip(mutual_information, indexes), reverse=True)
print('Sorted indexes: {}'.format(sorted_indexes[:4]))

# keep the indexes of the best k features
best_idxs = [idx for val, idx in sorted_indexes[:k]]
print('Best indexes: {}\n'.format(best_idxs[:4]))

# Keep only the colums of the best features
x_train_best_tfidf = x_train_tfidf_array[:, best_idxs]
x_test_best_tfidf = x_test_tfidf[:, best_idxs]

print('Shape of training data: {} --> {}'.format(x_train_tfidf.shape, x_train_best_tfidf.shape))
print('Shape of test data: {} --> {}\n'.format(x_test_tfidf.shape, x_test_best_tfidf.shape))

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.

Sorted indexes: [(0.6215552065711296, 1592), (0.6213756291496941, 3130), (0.5671079003385351, 2905), (0.527157711181147, 2561)]
Best indexes: [1592, 3130, 2905, 2561]

Shape of training data: (1400, 5000) --> (1400, 2500)
Shape of test data: (200, 5000) --> (200, 2500)



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dis

# Dummy Classifier

## Boolean Features

In [ ]:
from sklearn.dummy import DummyClassifier

# Train
# The dummy classifier always predicts the 'most frequent' class
baseline = DummyClassifier(strategy='most_frequent')
start_time = time.time()
baseline.fit(x_train_tfidf, y_train)
print("Training took: {} seconds \n".format(time.time() - start_time))

# Evaluate

# Train
predictions_train_dummy_boolean = baseline.predict(x_train_boolean)
precision_train_dummy_boolean, recall_train_dummy_boolean, f1_train_dummy_boolean, support_train_dummy_boolean = precision_recall_fscore_support(y_train, predictions_train_dummy_boolean)
macro_precision_train_dummy_boolean = (precision_train_dummy_boolean[0] + precision_train_dummy_boolean[1])/2
macro_recall_train_dummy_boolean = (recall_train_dummy_boolean[0] + recall_train_dummy_boolean[1])/2
macro_f1_train_dummy_boolean = (f1_train_dummy_boolean[0] + f1_train_dummy_boolean[1])/2
probs_y_train_dummy_boolean = baseline.predict_proba(x_train_boolean) 
precisions_train_pos_dummy_boolean, recalls_train_pos_dummy_boolean, thresholds_train_pos_dummy_boolean = precision_recall_curve(y_train, probs_y_train_dummy_boolean[:, 1], pos_label=1)
precisions_train_neg_dummy_boolean, recalls_train_neg_dummy_boolean, thresholds_train_neg_dummy_boolean = precision_recall_curve(y_train, probs_y_train_dummy_boolean[:, 1], pos_label=0) 
auc_train_pos_dummy_boolean = auc(recalls_train_pos_dummy_boolean, precisions_train_pos_dummy_boolean)
auc_train_neg_dummy_boolean = auc(recalls_train_neg_dummy_boolean, precisions_train_neg_dummy_boolean)
macro_auc_train_dummy_boolean = (auc_train_pos_dummy_boolean + auc_train_neg_dummy_boolean)/2


# Dev
predictions_dev_dummy_boolean = baseline.predict(x_dev_boolean)
precision_dev_dummy_boolean, recall_dev_dummy_boolean, f1_dev_dummy_boolean, support_dev_dummy_boolean = precision_recall_fscore_support(y_dev, predictions_dev_dummy_boolean)
macro_precision_dev_dummy_boolean = (precision_dev_dummy_boolean[0] + precision_dev_dummy_boolean[1])/2
macro_recall_dev_dummy_boolean = (recall_dev_dummy_boolean[0] + recall_dev_dummy_boolean[1])/2
macro_f1_dev_dummy_boolean = (f1_dev_dummy_boolean[0] + f1_dev_dummy_boolean[1])/2
probs_y_dev_dummy_boolean = baseline.predict_proba(x_dev_boolean) 
precisions_dev_pos_dummy_boolean, recalls_dev_pos_dummy_boolean, thresholds_dev_pos_dummy_boolean = precision_recall_curve(y_dev, probs_y_dev_dummy_boolean[:, 1], pos_label=1)
precisions_dev_neg_dummy_boolean, recalls_dev_neg_dummy_boolean, thresholds_dev_neg_dummy_boolean = precision_recall_curve(y_dev, probs_y_dev_dummy_boolean[:, 1], pos_label=0) 
auc_dev_pos_dummy_boolean = auc(recalls_dev_pos_dummy_boolean, precisions_dev_pos_dummy_boolean)
auc_dev_neg_dummy_boolean = auc(recalls_dev_neg_dummy_boolean, precisions_dev_neg_dummy_boolean)
macro_auc_dev_dummy_boolean = (auc_dev_pos_dummy_boolean + auc_dev_neg_dummy_boolean)/2


# Test
predictions_test_dummy_boolean = baseline.predict(x_test_boolean)
precision_test_dummy_boolean, recall_test_dummy_boolean, f1_test_dummy_boolean, support_test_dummy_boolean = precision_recall_fscore_support(y_test, predictions_test_dummy_boolean)
macro_precision_test_dummy_boolean = (precision_test_dummy_boolean[0] + precision_test_dummy_boolean[1])/2
macro_recall_test_dummy_boolean = (recall_test_dummy_boolean[0] + recall_test_dummy_boolean[1])/2
macro_f1_test_dummy_boolean = (f1_test_dummy_boolean[0] + f1_test_dummy_boolean[1])/2
probs_y_test_dummy_boolean = baseline.predict_proba(x_test_boolean) 
precisions_test_pos_dummy_boolean, recalls_test_pos_dummy_boolean, thresholds_test_pos_dummy_boolean = precision_recall_curve(y_test, probs_y_test_dummy_boolean[:, 1], pos_label=1)
precisions_test_neg_dummy_boolean, recalls_test_neg_dummy_boolean, thresholds_test_neg_dummy_boolean = precision_recall_curve(y_test, probs_y_test_dummy_boolean[:, 1], pos_label=0) 
auc_test_pos_dummy_boolean = auc(recalls_test_pos_dummy_boolean, precisions_test_pos_dummy_boolean)
auc_test_neg_dummy_boolean = auc(recalls_test_neg_dummy_boolean, precisions_test_neg_dummy_boolean)
macro_auc_test_dummy_boolean = (auc_test_pos_dummy_boolean + auc_test_neg_dummy_boolean)/2

NameError: ignored

## Tf-Idf Features

In [ ]:
# Train
# The dummy classifier always predicts the 'most frequent' class
baseline = DummyClassifier(strategy='most_frequent')
start_time = time.time()
baseline.fit(x_train_tfidf, y_train)
print("Training took: {} seconds \n".format(time.time() - start_time))

# Evaluate

# Train
predictions_train_dummy_tfidf = baseline.predict(x_train_tfidf)
precision_train_dummy_tfidf, recall_train_dummy_tfidf, f1_train_dummy_tfidf, support_train_dummy_tfidf = precision_recall_fscore_support(y_train, predictions_train_dummy_tfidf)
macro_precision_train_dummy_tfidf = (precision_train_dummy_tfidf[0] + precision_train_dummy_tfidf[1])/2
macro_recall_train_dummy_tfidf = (recall_train_dummy_tfidf[0] + recall_train_dummy_tfidf[1])/2
macro_f1_train_dummy_tfidf = (f1_train_dummy_tfidf[0] + f1_train_dummy_tfidf[1])/2
probs_y_train_dummy_tfidf = baseline.predict_proba(x_train_tfidf) 
precisions_train_pos_dummy_tfidf, recalls_train_pos_dummy_tfidf, thresholds_train_pos_dummy_tfidf = precision_recall_curve(y_train, probs_y_train_dummy_tfidf[:, 1], pos_label=1)
precisions_train_neg_dummy_tfidf, recalls_train_neg_dummy_tfidf, thresholds_train_neg_dummy_tfidf = precision_recall_curve(y_train, probs_y_train_dummy_tfidf[:, 1], pos_label=0) 
auc_train_pos_dummy_tfidf = auc(recalls_train_pos_dummy_tfidf, precisions_train_pos_dummy_tfidf)
auc_train_neg_dummy_tfidf = auc(recalls_train_neg_dummy_tfidf, precisions_train_neg_dummy_tfidf)
macro_auc_train_dummy_tfidf = (auc_train_pos_dummy_tfidf + auc_train_neg_dummy_tfidf)/2


# Dev
predictions_dev_dummy_tfidf = baseline.predict(x_dev_tfidf)
precision_dev_dummy_tfidf, recall_dev_dummy_tfidf, f1_dev_dummy_tfidf, support_dev_dummy_tfidf = precision_recall_fscore_support(y_dev, predictions_dev_dummy_tfidf)
macro_precision_dev_dummy_tfidf = (precision_dev_dummy_tfidf[0] + precision_dev_dummy_tfidf[1])/2
macro_recall_dev_dummy_tfidf = (recall_dev_dummy_tfidf[0] + recall_dev_dummy_tfidf[1])/2
macro_f1_dev_dummy_tfidf = (f1_dev_dummy_tfidf[0] + f1_dev_dummy_tfidf[1])/2
probs_y_dev_dummy_tfidf = baseline.predict_proba(x_dev_tfidf) 
precisions_dev_pos_dummy_tfidf, recalls_dev_pos_dummy_tfidf, thresholds_dev_pos_dummy_tfidf = precision_recall_curve(y_dev, probs_y_dev_dummy_tfidf[:, 1], pos_label=1)
precisions_dev_neg_dummy_tfidf, recalls_dev_neg_dummy_tfidf, thresholds_dev_neg_dummy_tfidf = precision_recall_curve(y_dev, probs_y_dev_dummy_tfidf[:, 1], pos_label=0) 
auc_dev_pos_dummy_tfidf = auc(recalls_dev_pos_dummy_tfidf, precisions_dev_pos_dummy_tfidf)
auc_dev_neg_dummy_tfidf = auc(recalls_dev_neg_dummy_tfidf, precisions_dev_neg_dummy_tfidf)
macro_auc_dev_dummy_tfidf = (auc_dev_pos_dummy_tfidf + auc_dev_neg_dummy_tfidf)/2


# Test
predictions_test_dummy_tfidf = baseline.predict(x_test_tfidf)
precision_test_dummy_tfidf, recall_test_dummy_tfidf, f1_test_dummy_tfidf, support_test_dummy_tfidf = precision_recall_fscore_support(y_test, predictions_test_dummy_tfidf)
macro_precision_test_dummy_tfidf = (precision_test_dummy_tfidf[0] + precision_test_dummy_tfidf[1])/2
macro_recall_test_dummy_tfidf = (recall_test_dummy_tfidf[0] + recall_test_dummy_tfidf[1])/2
macro_f1_test_dummy_tfidf = (f1_test_dummy_tfidf[0] + f1_test_dummy_tfidf[1])/2
probs_y_test_dummy_tfidf = baseline.predict_proba(x_test_tfidf) 
precisions_test_pos_dummy_tfidf, recalls_test_pos_dummy_tfidf, thresholds_test_pos_dummy_tfidf = precision_recall_curve(y_test, probs_y_test_dummy_tfidf[:, 1], pos_label=1)
precisions_test_neg_dummy_tfidf, recalls_test_neg_dummy_tfidf, thresholds_test_neg_dummy_tfidf = precision_recall_curve(y_test, probs_y_test_dummy_tfidf[:, 1], pos_label=0) 
auc_test_pos_dummy_tfidf = auc(recalls_test_pos_dummy_tfidf, precisions_test_pos_dummy_tfidf)
auc_test_neg_dummy_tfidf = auc(recalls_test_neg_dummy_tfidf, precisions_test_neg_dummy_tfidf)
macro_auc_test_dummy_tfidf = (auc_test_pos_dummy_tfidf + auc_test_neg_dummy_tfidf)/2

# Linear Regression

## Boolean Features

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, precision_recall_fscore_support, roc_auc_score, precision_recall_curve, auc
import time


clf = LogisticRegression(solver="saga", penalty='elasticnet', l1_ratio=0.5, C=0.5)
start_time = time.time()
clf.fit(x_train_boolean, y_train)
print("Training took: {} seconds \n".format(time.time() - start_time))

# Evaluate

# Train
predictions_train = clf.predict(x_train_boolean)
precision_train, recall_train, f1_train, support_train = precision_recall_fscore_support(y_train, predictions_train)
macro_precision_train = (precision_train[0] + precision_train[1])/2
macro_recall_train = (recall_train[0] + recall_train[1])/2
macro_f1_train = (f1_train[0] + f1_train[1])/2
probs_y_train=clf.predict_proba(x_train_boolean) 
precisions_train_pos, recalls_train_pos, thresholds_train_pos = precision_recall_curve(y_train, probs_y_train[:, 1], pos_label=1)
precisions_train_neg, recalls_train_neg, thresholds_train_neg = precision_recall_curve(y_train, probs_y_train[:, 1], pos_label=0) 
auc_train_pos = auc(recalls_train_pos, precisions_train_pos)
auc_train_neg = auc(recalls_train_neg, precisions_train_neg)
macro_auc_train = (auc_train_pos + auc_train_neg)/2


# Dev
predictions_dev = clf.predict(x_dev_boolean)
precision_dev, recall_dev, f1_dev, support_dev = precision_recall_fscore_support(y_dev, predictions_dev)
macro_precision_dev = (precision_dev[0] + precision_dev[1])/2
macro_recall_dev = (recall_dev[0] + recall_dev[1])/2
macro_f1_dev = (f1_dev[0] + f1_dev[1])/2
probs_y_dev=clf.predict_proba(x_dev_boolean) 
precisions_dev_pos, recalls_dev_pos, thresholds_dev_pos = precision_recall_curve(y_dev, probs_y_dev[:, 1], pos_label=1)
precisions_dev_neg, recalls_dev_neg, thresholds_dev_neg = precision_recall_curve(y_dev, probs_y_dev[:, 1], pos_label=0) 
auc_dev_pos = auc(recalls_dev_pos, precisions_dev_pos)
auc_dev_neg = auc(recalls_dev_neg, precisions_dev_neg)
macro_auc_dev = (auc_dev_pos + auc_dev_neg)/2


# Test
predictions_test = clf.predict(x_test_boolean)
precision_test, recall_test, f1_test, support_test = precision_recall_fscore_support(y_test, predictions_test)
macro_precision_test = (precision_test[0] + precision_test[1])/2
macro_recall_test = (recall_test[0] + recall_test[1])/2
macro_f1_test = (f1_test[0] + f1_test[1])/2
probs_y_test=clf.predict_proba(x_test_boolean) 
precisions_test_pos, recalls_test_pos, thresholds_test_pos = precision_recall_curve(y_test, probs_y_test[:, 1], pos_label=1)
precisions_test_neg, recalls_test_neg, thresholds_test_neg = precision_recall_curve(y_test, probs_y_test[:, 1], pos_label=0) 
auc_test_pos = auc(recalls_test_pos, precisions_test_pos)
auc_test_neg = auc(recalls_test_neg, precisions_test_neg)
macro_auc_test = (auc_test_pos + auc_test_neg)/2

## Tf-Idf Features

In [ ]:
clf = LogisticRegression(solver="saga", penalty='elasticnet', l1_ratio=0.5, C=0.5)
start_time = time.time()
clf.fit(x_train_tfidf, y_train)
print("Training took: {} seconds \n".format(time.time() - start_time))

# Evaluate

# Train
predictions_train_tfidf = clf.predict(x_train_tfidf)
precision_train_tfidf, recall_train_tfidf, f1_train_tfidf, support_train_tfidf = precision_recall_fscore_support(y_train, predictions_train_tfidf)
macro_precision_train_tfidf = (precision_train_tfidf[0] + precision_train_tfidf[1])/2
macro_recall_train_tfidf = (recall_train_tfidf[0] + recall_train_tfidf[1])/2
macro_f1_train_tfidf = (f1_train_tfidf[0] + f1_train_tfidf[1])/2
probs_y_train_tfidf=clf.predict_proba(x_train_tfidf) 
precisions_train_pos_tfidf, recalls_train_pos_tfidf, thresholds_train_pos_tfidf = precision_recall_curve(y_train, probs_y_train_tfidf[:, 1], pos_label=1)
precisions_train_neg_tfidf, recalls_train_neg_tfidf, thresholds_train_neg_tfidf = precision_recall_curve(y_train, probs_y_train_tfidf[:, 1], pos_label=0) 
auc_train_pos_tfidf = auc(recalls_train_pos_tfidf, precisions_train_pos_tfidf)
auc_train_neg_tfidf = auc(recalls_train_neg_tfidf, precisions_train_neg_tfidf)
macro_auc_train_tfidf = (auc_train_pos_tfidf + auc_train_neg_tfidf)/2


# Dev
predictions_dev_tfidf = clf.predict(x_dev_tfidf)
precision_dev_tfidf, recall_dev_tfidf, f1_dev_tfidf, support_dev_tfidf = precision_recall_fscore_support(y_dev, predictions_dev_tfidf)
macro_precision_dev_tfidf = (precision_dev_tfidf[0] + precision_dev_tfidf[1])/2
macro_recall_dev_tfidf = (recall_dev_tfidf[0] + recall_dev_tfidf[1])/2
macro_f1_dev_tfidf = (f1_dev_tfidf[0] + f1_dev_tfidf[1])/2
probs_y_dev_tfidf=clf.predict_proba(x_dev_tfidf) 
precisions_dev_pos_tfidf, recalls_dev_pos_tfidf, thresholds_dev_pos_tfidf = precision_recall_curve(y_dev, probs_y_dev_tfidf[:, 1], pos_label=1)
precisions_dev_neg_tfidf, recalls_dev_neg_tfidf, thresholds_dev_neg_tfidf = precision_recall_curve(y_dev, probs_y_dev_tfidf[:, 1], pos_label=0) 
auc_dev_pos_tfidf = auc(recalls_dev_pos_tfidf, precisions_dev_pos_tfidf)
auc_dev_neg_tfidf = auc(recalls_dev_neg_tfidf, precisions_dev_neg_tfidf)
macro_auc_dev_tfidf = (auc_dev_pos_tfidf + auc_dev_neg_tfidf)/2


# Test
predictions_test_tfidf = clf.predict(x_test_tfidf)
precision_test_tfidf, recall_test_tfidf, f1_test_tfidf, support_test_tfidf = precision_recall_fscore_support(y_test, predictions_test_tfidf)
macro_precision_test_tfidf = (precision_test_tfidf[0] + precision_test_tfidf[1])/2
macro_recall_test_tfidf = (recall_test_tfidf[0] + recall_test_tfidf[1])/2
macro_f1_test_tfidf = (f1_test_tfidf[0] + f1_test_tfidf[1])/2
probs_y_test_tfidf=clf.predict_proba(x_test_tfidf) 
precisions_test_pos_tfidf, recalls_test_pos_tfidf, thresholds_test_pos_tfidf = precision_recall_curve(y_test, probs_y_test_tfidf[:, 1], pos_label=1)
precisions_test_neg_tfidf, recalls_test_neg_tfidf, thresholds_test_neg_tfidf = precision_recall_curve(y_test, probs_y_test_tfidf[:, 1], pos_label=0) 
auc_test_pos_tfidf = auc(recalls_test_pos_tfidf, precisions_test_pos_tfidf)
auc_test_neg_tfidf = auc(recalls_test_neg_tfidf, precisions_test_neg_tfidf)
macro_auc_test_tfidf = (auc_test_pos_tfidf + auc_test_neg_tfidf)/2

# Scores for Train

In [ ]:
import pandas as pd
import matplotlib as mpl

df = pd.DataFrame([[precision_train_dummy_boolean[0], precision_train_dummy_boolean[1], recall_train_dummy_boolean[0], recall_train_dummy_boolean[1], f1_train_dummy_boolean[0], f1_train_dummy_boolean[1], auc_train_pos_dummy_boolean, auc_train_neg_dummy_boolean, macro_precision_train_dummy_boolean, macro_precision_train_dummy_boolean, macro_recall_train_dummy_boolean, macro_recall_train_dummy_boolean, macro_f1_train_dummy_boolean, macro_f1_train_dummy_boolean, macro_auc_train_dummy_boolean, macro_auc_train_dummy_boolean], 
                   [precision_train_dummy_tfidf[0], precision_train_dummy_tfidf[1], recall_train_dummy_tfidf[0], recall_train_dummy_tfidf[1], f1_train_dummy_tfidf[0], f1_train_dummy_tfidf[1], auc_train_pos_dummy_tfidf, auc_train_neg_dummy_tfidf, macro_precision_train_dummy_tfidf, macro_precision_train_dummy_tfidf, macro_recall_train_dummy_tfidf, macro_recall_train_dummy_tfidf, macro_f1_train_dummy_tfidf, macro_f1_train_dummy_tfidf, macro_auc_train_dummy_tfidf, macro_auc_train_dummy_tfidf], 
                   [precision_train[0], precision_train[1], recall_train[0], recall_train[1], f1_train[0], f1_train[1], auc_train_pos, auc_train_neg, macro_precision_train, macro_precision_train, macro_recall_train, macro_recall_train, macro_f1_train, macro_f1_train, macro_auc_train, macro_auc_train], 
                   [precision_train_tfidf[0], precision_train_tfidf[1], recall_train_tfidf[0], recall_train_tfidf[1], f1_train_tfidf[0], f1_train_tfidf[1], auc_train_pos_tfidf, auc_train_neg_tfidf, macro_precision_train_tfidf, macro_precision_train_tfidf, macro_recall_train_tfidf, macro_recall_train_tfidf, macro_f1_train_tfidf, macro_f1_train_tfidf, macro_auc_train_tfidf, macro_auc_train_tfidf]],
                  index=pd.Index(['Dummy Classifier (Boolean)', 'Dummy Classifier (Tf-Idf)', 'Logistic Regression (Boolean)', 'Logistic Regression (Tf-Idf)'], name='Model:'),
                  columns=pd.MultiIndex.from_product([['Precision', 'Recall', 'F1', 'AUC', 'Macro Precision', 'Macro Recall', 'Macro F1', 'Macro AUC'],['Positive', 'Negative']], names=['Scores:', 'Class:']))
df.style

# Scores for Development

In [ ]:
df = pd.DataFrame([[precision_dev_dummy_boolean[0], precision_dev_dummy_boolean[1], recall_dev_dummy_boolean[0], recall_dev_dummy_boolean[1], f1_dev_dummy_boolean[0], f1_dev_dummy_boolean[1], auc_dev_pos_dummy_boolean, auc_dev_neg_dummy_boolean, macro_precision_dev_dummy_boolean, macro_precision_dev_dummy_boolean, macro_recall_dev_dummy_boolean, macro_recall_dev_dummy_boolean, macro_f1_dev_dummy_boolean, macro_f1_dev_dummy_boolean, macro_auc_dev_dummy_boolean, macro_auc_dev_dummy_boolean], 
                   [precision_dev_dummy_tfidf[0], precision_dev_dummy_tfidf[1], recall_dev_dummy_tfidf[0], recall_dev_dummy_tfidf[1], f1_dev_dummy_tfidf[0], f1_dev_dummy_tfidf[1], auc_dev_pos_dummy_tfidf, auc_dev_neg_dummy_tfidf, macro_precision_dev_dummy_tfidf, macro_precision_dev_dummy_tfidf, macro_recall_dev_dummy_tfidf, macro_recall_dev_dummy_tfidf, macro_f1_dev_dummy_tfidf, macro_f1_dev_dummy_tfidf, macro_auc_dev_dummy_tfidf, macro_auc_dev_dummy_tfidf], 
                   [precision_dev[0], precision_dev[1], recall_dev[0], recall_dev[1], f1_dev[0], f1_dev[1], auc_dev_pos, auc_dev_neg, macro_precision_dev, macro_precision_dev, macro_recall_dev, macro_recall_dev, macro_f1_dev, macro_f1_dev, macro_auc_dev, macro_auc_dev], 
                   [precision_dev_tfidf[0], precision_dev_tfidf[1], recall_dev_tfidf[0], recall_dev_tfidf[1], f1_dev_tfidf[0], f1_dev_tfidf[1], auc_dev_pos_tfidf, auc_dev_neg_tfidf, macro_precision_dev_tfidf, macro_precision_dev_tfidf, macro_recall_dev_tfidf, macro_recall_dev_tfidf, macro_f1_dev_tfidf, macro_f1_dev_tfidf, macro_auc_dev_tfidf, macro_auc_dev_tfidf]],
                  index=pd.Index(['Dummy Classifier (Boolean)', 'Dummy Classifier (Tf-Idf)', 'Logistic Regression (Boolean)', 'Logistic Regression (Tf-Idf)'], name='Model:'),
                  columns=pd.MultiIndex.from_product([['Precision', 'Recall', 'F1', 'AUC', 'Macro Precision', 'Macro Recall', 'Macro F1', 'Macro AUC'],['Positive', 'Negative']], names=['Scores:', 'Class:']))
df.style

# Scores for Test

In [ ]:
df = pd.DataFrame([[precision_test_dummy_boolean[0], precision_test_dummy_boolean[1], recall_test_dummy_boolean[0], recall_test_dummy_boolean[1], f1_test_dummy_boolean[0], f1_test_dummy_boolean[1], auc_test_pos_dummy_boolean, auc_test_neg_dummy_boolean, macro_precision_test_dummy_boolean, macro_precision_test_dummy_boolean, macro_recall_test_dummy_boolean, macro_recall_test_dummy_boolean, macro_f1_test_dummy_boolean, macro_f1_test_dummy_boolean, macro_auc_test_dummy_boolean, macro_auc_test_dummy_boolean], 
                   [precision_test_dummy_tfidf[0], precision_test_dummy_tfidf[1], recall_test_dummy_tfidf[0], recall_test_dummy_tfidf[1], f1_test_dummy_tfidf[0], f1_test_dummy_tfidf[1], auc_test_pos_dummy_tfidf, auc_test_neg_dummy_tfidf, macro_precision_test_dummy_tfidf, macro_precision_test_dummy_tfidf, macro_recall_test_dummy_tfidf, macro_recall_test_dummy_tfidf, macro_f1_test_dummy_tfidf, macro_f1_test_dummy_tfidf, macro_auc_test_dummy_tfidf, macro_auc_test_dummy_tfidf],
                   [precision_test[0], precision_test[1], recall_test[0], recall_test[1], f1_test[0], f1_test[1], auc_test_pos, auc_test_neg, macro_precision_test, macro_precision_test, macro_recall_test, macro_recall_test, macro_f1_test, macro_f1_test, macro_auc_test, macro_auc_test], 
                   [precision_test_tfidf[0], precision_test_tfidf[1], recall_test_tfidf[0], recall_test_tfidf[1], f1_test_tfidf[0], f1_test_tfidf[1], auc_test_pos_tfidf, auc_test_neg_tfidf, macro_precision_test_tfidf, macro_precision_test_tfidf, macro_recall_test_tfidf, macro_recall_test_tfidf, macro_f1_test_tfidf, macro_f1_test_tfidf, macro_auc_test_tfidf, macro_auc_test_tfidf]],
                  index=pd.Index(['Dummy Classifier (Boolean)', 'Dummy Classifier (Tf-Idf)', 'Logistic Regression (Boolean)', 'Logistic Regression (Tf-Idf)'], name='Model:'),
                  columns=pd.MultiIndex.from_product([['Precision', 'Recall', 'F1', 'AUC', 'Macro Precision', 'Macro Recall', 'Macro F1', 'Macro AUC'],['Positive', 'Negative']], names=['Scores:', 'Class:']))
df.style

# Learning Curves

In [ ]:
# Learning curves with cross-validtion
import numpy as np
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression

#linspace returns num evenly spaced samples, calculated over the interval [start, stop].
def plot_learning_curve(estimator, title, X, y, dev_x, dev_y, test_x, test_y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
   
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("macro F1-score")
    
    train_sizes1, train_scores, cv_train_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs,scoring='f1_macro', train_sizes=train_sizes)
   
    
    train_sizes2, dev_scores, cv_dev_scores = learning_curve(
        estimator, dev_x, dev_y,cv=cv, n_jobs=n_jobs,scoring='f1_macro', train_sizes=train_sizes)
    
    train_sizes3, test_scores, cv_test_scores = learning_curve(
        estimator, test_x, test_y,cv=cv, n_jobs=n_jobs,scoring='f1_macro', train_sizes=train_sizes)

    cv_train_scores_mean = np.mean(cv_train_scores, axis=1)
    cv_train_scores_std = np.std(cv_train_scores, axis=1)

    cv_dev_scores_mean = np.mean(cv_dev_scores, axis=1)
    cv_dev_scores_std = np.std(cv_dev_scores, axis=1)
    
    cv_test_scores_mean = np.mean(cv_test_scores, axis=1)
    cv_test_scores_std = np.std(cv_test_scores, axis=1)
   
    plt.grid()

    plt.fill_between(train_sizes1, cv_train_scores_mean -  cv_train_scores_std,
                     cv_train_scores_mean +  cv_train_scores_std, alpha=0.1,
                     color="b")
    plt.fill_between(train_sizes2, cv_test_scores_mean - cv_test_scores_std,
                     cv_test_scores_mean + cv_test_scores_std, alpha=0.1, color="orange")
    
    plt.fill_between(train_sizes3, cv_dev_scores_mean - cv_dev_scores_std,
                     cv_dev_scores_mean + cv_dev_scores_std, alpha=0.1, color="red")
    
    plt.plot(train_sizes1, cv_train_scores_mean, 'o-', color="b",
             label="Train score")
    plt.plot(train_sizes2, cv_test_scores_mean, 'o-', color="orange",
             label="Test score")
    plt.plot(train_sizes3, cv_dev_scores_mean, 'o-', color="red",
             label="Dev score")

    plt.legend(loc="lower right")
    return plt

# Use cross validation with 5 splits, 20% test subset size each time


title = "Learning Curves (Logistic Regression (Tf-Idf))"
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
estimator = LogisticRegression(solver="saga", penalty='elasticnet', l1_ratio=0.5, C=0.5)
plot_learning_curve(estimator, title, x_train_tfidf, y_train, x_dev_tfidf, y_dev, x_test_tfidf, y_test,(0.1, 1.01), cv=cv, n_jobs=-1)



title = "Learning Curves (Logistic Regression (Boolean))"
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
estimator = LogisticRegression(solver="saga", penalty='elasticnet', l1_ratio=0.5, C=0.5)
plot_learning_curve(estimator, title, x_train_boolean, y_train, x_dev_boolean, y_dev, x_test_boolean, y_test,(0.1, 1.01), cv=cv, n_jobs=-1)


title = "Learning Curves (DummyClassifier (Tf-Idf))"
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
estimator = DummyClassifier(strategy='most_frequent')
plot_learning_curve(estimator, title, x_train_tfidf, y_train, x_dev_tfidf, y_dev, x_test_tfidf, y_test,(0.1, 1.01), cv=cv, n_jobs=-1)



title = "Learning Curves (DummyClassifier(Boolean))"
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
estimator = DummyClassifier(strategy='most_frequent')
plot_learning_curve(estimator, title, x_train_boolean, y_train, x_dev_boolean, y_dev, x_test_boolean, y_test,(0.1, 1.01), cv=cv, n_jobs=-1)
plt.show()